# Integrating your Devices with MADSci Nodes

This notebook aims to teach you how to automate and integrate all the devices, instruments, sensors, and robots in your self-driving lab using the MADSci Node standard.

## Goals

After completing this notebook, you should understand

1. What we mean when we talk about a MADSci Node
2. The MADSci Node Standard
3. How to integrate and automate a device using the MADSci Node standard
4. How to use the `RestNode` python class included in `madsci.node_module` to integrate a MADSci Node


## What is a Node?

- In MADSci, a `Node` refers to a single instrument, sensor, robot, or other device, combined with the software needed to control, operate, automate, and integrate it into the Automated or Autonomous Lab as a whole.
- Node's take **Action Requests**, and return **Action Results**. They also report **State** and **Status**, **Node Info**, and can optionally manage **Resources**.

### Anatomy of a Node

A `Node` typically consists of the following sub-components:

- A physical **device** (robot, instrument, sensor, etc.)
- A **driver**, API, library, or software application for communicating with that device, often provided by the hardware vendor
- A device **interface** class that handles the neccessary initialization, communication, and cleanup required to use the given device
- The **node implementation**, typically a class, which uses the interface to handle the execution of the actions and the lifecycle of the node (state, status, statup, shutdown, etc.).
- The **node definition** and **configuration**, which define specific details about a given instance of a node and how it should. We typically implement these as .YAML files
- The **node server** and **node client**, which allow for standardized control of nodes. These are implementations of the MADSci Node standard interface, and operate using standard protocols (currently, REST-based HTTP)

<img src="./assets/mermaid/NodeDiagram.svg" width=1000></img>

### Node Instances vs. Node Modules

Sometimes, we need to disambiguate between "this particular Node, tied to a specific device" and "this class of Node, tied to this type of device". We usually refer to the former as a **Node Instance**, and the latter as a **Node Module** when we need to be precise.

For example, we might have one "Opentrons OT-2" **Node Module** that we've written, which we then use to run 2 different **Node Instances** in the lab: "OT-2 Alpha" and "OT-2 Beta"

In [27]:
# Install dependencies
%pip install madsci.common madsci.node_module httpx

/Users/luckierdodge/AD-SDL/MADSci_Examples/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


## Defining A Node

While you can build a MADSci compliant Node entirely from scratch, that's a lot of work. To save you some time and effort, we've implemented our own `RestNode` python class that you can inherit from as the basis for defining a device's Node.

In the rest of the notebook, we'll demonstrate an example of integrating a fake "robot arm" as a MADSci Node Module using our `RestNode` class.

### Node Definition Files

In addition to the code that makes up a Node Module's implementation, an individual instance of a Node has a `NodeDefinition`, which includes all the configuration and information for a **single, specific instance** of a device. We generally store this definition as a `.yaml` file that is then passed as an argument when we start the Node server.

These `NodeDefinitions` allow you to easily differentiate between and configure multiple **Node Instances** of a given **Node Module**

In [28]:
from madsci.node_module.rest_node_module import RestNode
from madsci.common.types.node_types import NodeDefinition, NodeType
from rich import print

In [29]:
class ExampleRobotNode(RestNode):
    """Define an Example Robot Node. It doesn't do anything yet, but it's a good starting point."""

    pass

# if __name__ == "__main__":
#     example_node = ExampleRobotNode()
#     example_node.start_node()
#
# Then, run `python example_robot_node.py --definition <path/to/example_robot.node.yaml` to start the node

In [30]:
# In most cases, this node definition is a .yaml file that we pass as the "--definition" argument at runtime,
# rather than defining it in code.
node_definition = NodeDefinition(
    node_name="example_robot_1",
    module_name="example_robot_module",
    node_type=NodeType.DEVICE,
    node_description="An example node for controlling our fake robot arm.",
)
example_node = ExampleRobotNode(node_definition=node_definition)
# Normally, `start_node` starts a http server and begins listening for incoming requests.
# However, in this case, we are using the `testing` argument to "start the node" without actually starting the server.
example_node.start_node(testing=True)

print(node_definition)

NodeDefinition(
    node_name='example_robot_1',
    node_id='01JRBCQV3MGHTHNSTF3JNH6NBN',
    node_url=None,
    node_description='An example node for controlling our fake robot arm.',
    node_type=<NodeType.DEVICE: 'device'>,
    module_name='example_robot_module',
    module_version=Version(major=0, minor=0, patch=1, prerelease=None, build=None),
    capabilities=NodeCapabilities(
        get_info=True,
        get_state=True,
        get_status=True,
        send_action=True,
        get_action_result=True,
        get_action_history=True,
        action_files=True,
        send_admin_commands=True,
        set_config=True,
        get_resources=False,
        get_log=True,
        events=None,
        resources=None,
        admin_commands={
            <AdminCommands.LOCK: 'lock'>,
            <AdminCommands.RESET: 'reset'>,
            <AdminCommands.UNLOCK: 'unlock'>,
            <AdminCommands.SHUTDOWN: 'shutdown'>
        }
    ),
    commands={},
    is_template=False,
    config_defaults={}
)

In [31]:
# Demo Magic to avoid having to actually run rest servers
# This is a mock server that simulates the behavior of the actual server.
# It allows us to test the client without needing to run the server.
# This is useful for unit testing and debugging, or running a demo
import contextlib
from fastapi.testclient import TestClient

from collections.abc import Generator
from typing import Any
from unittest.mock import patch

from madsci.client.node.rest_node_client import RestNodeClient

@contextlib.contextmanager
def node_server(node: RestNode) -> Generator[TestClient, None, None]:
    """Mock server context manager."""

    test_client = TestClient(node.rest_api)

    with test_client as requests:
        # Mock the server's behavior
        yield requests

@contextlib.contextmanager
def node_client(node: RestNode, client: RestNodeClient) -> Generator[RestNodeClient, None, None]:
    """Mock client context manager."""

    with node_server(node) as requests, patch("madsci.client.node.rest_node_client.requests") as mock_requests:
            def post_no_timeout(*args: Any, **kwargs: Any) -> Any:
                kwargs.pop("timeout", None)
                return requests.post(*args, **kwargs)

            mock_requests.post.side_effect = post_no_timeout

            def get_no_timeout(*args: Any, **kwargs: Any) -> Any:
                kwargs.pop("timeout", None)
                return requests.get(*args, **kwargs)

            mock_requests.get.side_effect = get_no_timeout

            yield client


## Node Info

> A node's way of introducing itself

A useful part of the MADSci Node Standard is the `NodeInfo`: a description of the node, it's current configuration and capabilites, and just about everything else you might reasonably want to know about it. The `RestNode` python class does it's best to automatically generate as much of the Node Info as possible, so that users, agents, and other components of the system can easily understand what the capabilities and functionality of a given node. 

In [5]:
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    # Example of a request to the node
    print(client.get_info())

2025-04-08 10:32:14.936599 (EventType.LOG_INFO): Startup complete for node example_node.

NodeInfo(
    node_name='example_node',
    node_id='01JRB0FNV771F9MWDPZ4GX31A7',
    node_url=AnyUrl('http://127.0.0.1:8000/'),
    node_description='An example node',
    node_type=None,
    module_name='example_node_module',
    module_version=SemanticVersion(major=0, minor=0, patch=1, prerelease=None, build=None),
    capabilities=NodeCapabilities(
        get_info=True,
        get_state=True,
        get_status=True,
        send_action=True,
        get_action_result=True,
        get_action_history=True,
        action_files=True,
        send_admin_commands=True,
        set_config=True,
        get_resources=False,
        get_log=True,
        events=None,
        resources=None,
        admin_commands={
            <AdminCommands.LOCK: 'lock'>,
            <AdminCommands.RESET: 'reset'>,
            <AdminCommands.UNLOCK: 'unlock'>,
            <AdminCommands.SHUTDOWN: 'shutdown'>
        }
    ),
    commands={},
    is_template=False,
    config_defaults={},
    actions={},
    config={
        'status_update_interval': 2.0,
        'state_update_interval': 2.0,
        'event_client_config': None,
        'resource_server_url': None,
        'host': '127.0.0.1',
        'port': 8000,
        'protocol': 'http'
    },
    config_schema={
        '$defs': {
            'EventClientConfig': {
                'description': 'Configuration for an Event Client.',
                'properties': {
                    'name': {
                        'anyOf': [{'type': 'string'}, {'type': 'null'}],
                        'default': None,
                        'description': 'The name of the event client.',
                        'title': 'Event Client Name'
                    },
                    'event_server_url': {
                        'anyOf': [{'type': 'string'}, {'type': 'null'}],
                        'default': None,
                        'description': 'The URL of the event server.',
                        'title': 'Event Server URL'
                    },
                    'log_level': {
                        'default': 20,
                        'description': 'The log level of the event client.',
                        'title': 'Event Client Log Level',
                        'type': 'integer'
                    },
                    'source': {
                        '$ref': '#/$defs/OwnershipInfo',
                        'description': 'Information about the source of the event client.',
                        'title': 'Source'
                    },
                    'log_dir': {
                        'anyOf': [{'type': 'string'}, {'format': 'path', 'type': 'string'}],
                        'description': 'The directory to store logs in.',
                        'title': 'Log Directory'
                    }
                },
                'title': 'EventClientConfig',
                'type': 'object'
            },
            'OwnershipInfo': {
                'description': 'Information about the ownership of a MADSci object.',
                'properties': {
                    'user_id': {
                        'anyOf': [{'type': 'string'}, {'type': 'null'}],
                        'default': None,
                        'description': 'The ID of the user who owns the object.',
                        'title': 'User ID'
                    },
                    'experiment_id': {
                        'anyOf': [{'type': 'string'}, {'type': 'null'}],
                        'default': None,
                        'description': 'The ID of the experiment that owns the object.',
                        'title': 'Experiment ID'
                    },
                    'campaign_id': {
                        'anyOf': [{'type': 'string'}, {'type': 'null'}],
                        'default': None,
                        'description': 'The ID of the campaign that owns the object.',
                        'title': 'Campaign ID'
                    },
        

## Node Configuration and Device Interfaces

In addition to the Node Class and the `NodeDefinition`, there are two other important things we may want to define in order to integrate a device.

### Node Config

The `NodeConfig` class is a class we define for a Node that, as the name implies, defines the configuration available for a specific Node. This often includes things like device identifiers or IP addresses, initialization arguments to pass to the device's driver, or details about the specific device being controlled. It also includes common configurations for the node itself, such as how to configure the REST server for the node.

We define our `NodeConfig` as a Pydantic Dataclass, and have a `RestNodeConfig` we can use as a base class with default configuration for the REST server.

One of the advantages of defining our `NodeConfig` in this way is that we can set configuration values:

- As part of the `NodeDefinition`, using the `config_overrides` field
- Using command-line parameters, when we start the Node
- Using the `set_config` endpoint on a running Node.

### Device Interface

Many of the devices we need to automate have their own DLL's, API's, Libraries, or other means of controlling and communicating with them. The exact details, capabilities, and implementations vary wildly from device to device, but in most cases these are complex enough to justify their own wrapper class.

Implementing the device specific logic as a standalone class provides two key advantages:

- We can use the device interface standalone, for testing and debugging, maintenance, or in applications where we can't or don't want to use a full Node.
- We separate the concerns of managing the node from the nuts and bolts of interfacing with the device.

This isn't always neccessary (some devices have API's so simple it simply isn't worth the effort), but is often worthwhile for complex devices or unwieldy API's.

In [32]:
from madsci.common.types.node_types import RestNodeConfig

class ExampleRobotConfig(RestNodeConfig):
    """Example Configuration options for our ExampleRobotNode."""

    robot_number: int = 0
    """An identifier for the robot we are controlling."""


In [33]:
import time

class ExampleRobotInterface:
    """Example Robot Interface. This is a simple interface for controlling a (fake) robot arm."""

    robot_number: int = 0
    """An identifier for the robot we are controlling."""
    joint_angles: list[float] = [0.0, 0.0, 0.0, 0.0]
    """The joint angles of the robot."""
    gripper_closed: bool = False
    """The state of the gripper, open or closed."""
    is_moving: bool = False
    """Whether the robot is currently moving."""

    def __init__(self, robot_number: int = 0):
        self.robot_number = robot_number

    def get_robot_number(self) -> int:
        """Get the robot number."""
        return self.robot_number

    def move_to_joint_angles(self, angles: list[float]) -> None:
        """Move the robot to the specified joint angles."""
        if self.is_moving:
            raise RuntimeError("Robot is already moving.")
        if len(angles) != 4:
            raise ValueError("Expected 4 joint angles.")
        self.is_moving = True
        time.sleep(5)
        self.is_moving = False
        self.joint_angles = angles

    def close_gripper(self) -> None:
        """Close the gripper."""
        if self.is_moving:
            raise RuntimeError("Robot is already moving.")
        self.is_moving = True
        time.sleep(1)
        self.is_moving = False
        self.gripper_closed = True

    def open_gripper(self) -> None:
        """Open the gripper."""
        if self.is_moving:
            raise RuntimeError("Robot is already moving.")
        self.is_moving = True
        time.sleep(1)
        self.is_moving = False
        self.gripper_closed = False

## Node Lifecycle

Generally speaking, nodes have the following common lifecycle components:

- `start_node`: this default function kickoffs the REST server, reads in the node definition and config, and calls the user defined `startup_handler`
- `startup_handler`: a user-defined function that handles whatever logic needs to be done to initialize the underlying device and prepare the node for operation. The node won't start accepting actions until the startup_handler completes.
- `shutdown_handler`: a user-defined function that handles the logic of shutting the node down: disconnecting from the device, cleaning up system resources, etc.
- `state_handler`: allows the node to update it's published state, a JSON serializable dictionary
- `status_handler`: allows the node to implement any custom logic around updating the `NodeStatus` used by the node and the rest of the system to understand the condition of the node.

The `startup_handler` and `shutdown_handler` generally run once, while the `state_handler` and `status_handler` run periodically, with a configurable interval.

<img src="./assets/drawio/node_lifecycle.drawio.svg" alt="Node Lifecycle" width=1000></img>

_Green represents user-defined functionality, blue is built-in_


In [34]:
from typing import Optional


class RobotNodeWithLifecycle(ExampleRobotNode):
    """Define an Example Node with a startup and shutdown handlers."""

    config_model = ExampleRobotConfig
    """The configuration model for the node."""
    robot_interface: Optional[ExampleRobotInterface] = None
    """The robot interface for controlling the robot."""

    def startup_handler(self) -> None:
        """Handle the startup of the node."""
        self.logger.log_info(f"Connecting to robot {self.config.robot_number}...")
        self.robot_interface = ExampleRobotInterface(self.config.robot_number)
        self.logger.log_info(f"Connected to robot {self.robot_interface.get_robot_number()}")

    def shutdown_handler(self) -> None:
        """Handle the shutdown of the node."""
        self.logger.log_info(f"Disconnecting from robot {self.config.robot_number}...")
        del self.robot_interface
        self.logger.log_info(f"Disconnected from robot {self.config.robot_number}")


In [35]:
example_node = RobotNodeWithLifecycle(node_definition=node_definition)
example_node.start_node(testing=True)

with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    while not client.get_status().ready:
        # Wait for the node to be ready
        time.sleep(0.1)
    print(client.get_status())

2025-04-08 14:14:32.241301 (EventType.LOG_INFO): Connecting to robot 0...

2025-04-08 14:14:32.268017 (EventType.LOG_INFO): Connected to robot 0

2025-04-08 14:14:32.270551 (EventType.LOG_INFO): Startup complete for node example_robot_1.

NodeStatus(
    busy=False,
    running_actions=set(),
    paused=False,
    locked=False,
    stopped=False,
    errored=False,
    errors=[],
    initializing=False,
    waiting_for_config=set(),
    config_values={},
    ready=True,
    description='Node is ready'
)

2025-04-08 14:14:32.378463 (EventType.LOG_INFO): Disconnecting from robot 0...

2025-04-08 14:14:32.379589 (EventType.LOG_INFO): Disconnected from robot 0

In [36]:
import time

class RobotNodeWithUpdates(RobotNodeWithLifecycle):
    """Define an Example Node that periodically updates it's status and public-facing state."""

    node_state = {
        "joint_angles": [0.0, 0.0, 0.0, 0.0],
    }

    def state_handler(self) -> None:
        """This is where you can implement logic to periodically update the node's public-facing state information."""
        if self.robot_interface is not None:
            self.node_state = {"joint_angles": self.robot_interface.joint_angles}
        else:
            self.node_state = {"joint_angles": None}

    def status_handler(self) -> None:
        """
        This is where you can implement logic to periodically update the node's status information.
        """
        if self.robot_interface is not None and self.robot_interface.is_moving:
            self.node_status.busy = True
        else:
            self.node_status.busy = len(self.node_status.running_actions) > 0


node_definition.config_defaults={
        "state_update_interval": 2, # Change how frequently, in seconds, the node state is updated
        "status_update_interval": 0.5, # Change how frequently, in seconds, the node status is updated
    }
example_node = RobotNodeWithUpdates(node_definition=node_definition)
example_node.node_status.errors = []
example_node.start_node(testing=True)

with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    while not client.get_status().ready:
        # Wait for the node to be ready
        time.sleep(0.1)
    print(client.get_state())
    print(client.get_status())
    example_node.robot_interface.move_to_joint_angles([0.5, 0.5, 0.5, 0.5])
    time.sleep(2.0) # -> Uncomment this line to wait for the updated state & status to be set
    print(client.get_state())
    print(client.get_status())

2025-04-08 14:15:53.403702 (EventType.LOG_INFO): Connecting to robot 0...

2025-04-08 14:15:53.406696 (EventType.LOG_INFO): Connected to robot 0

2025-04-08 14:15:53.407666 (EventType.LOG_INFO): Startup complete for node example_robot_1.

{'joint_angles': [0.0, 0.0, 0.0, 0.0]}

NodeStatus(
    busy=False,
    running_actions=set(),
    paused=False,
    locked=False,
    stopped=False,
    errored=False,
    errors=[],
    initializing=False,
    waiting_for_config=set(),
    config_values={},
    ready=True,
    description='Node is ready'
)

{'joint_angles': [0.5, 0.5, 0.5, 0.5]}

NodeStatus(
    busy=False,
    running_actions=set(),
    paused=False,
    locked=False,
    stopped=False,
    errored=False,
    errors=[],
    initializing=False,
    waiting_for_config=set(),
    config_values={},
    ready=True,
    description='Node is ready'
)

2025-04-08 14:16:00.527620 (EventType.LOG_INFO): Disconnecting from robot 0...

2025-04-08 14:16:00.529056 (EventType.LOG_INFO): Disconnected from robot 0

## Node Actions

Now that we have the lifecycle of the Node working, let's talk about **Actions**. In MADSci, a Node Action is, essentially, a function that you can call on the node in a standardized way. These functions typically, but not neccessarily, involve the device the node controls taking some action (e.g. a camera taking a picture, a robot arm moving, a plate reader doing a reading, etc.).

### Anatomy of an Action

Node Actions can be defined with the following:

- **Name**: a unique name for the action to perform
- **Description**: a human-readable description of the action
- **Arguments**: the arguments the action takes. Normal arguments must be JSON-serializable types, and can be either required or optional. There are two special cases of arguments:
    - **File Arguments**: files the action accepts as inputs, which are uploaded by the client
    - **Location Arguments**: these are functionally the same as regular Arguments, but have special meaning when using a node as part of a MADSci workcell (more on that later)
- **Results**: these are what an action returns after completing, typically in the form of JSON Data, Files, or Datapoint IDs

To perform an action with a node, you send an **ActionRequest**, and the node returns **ActionResults** capturing the progress and final outcomes of the result.

TODO: action endpoints diagram goes here

### Client-Server Action Methods

<img  src="./assets/drawio/action_methods.drawio.svg" style="background-color: white" width=1000 alt="Node Action Method/Endpoint Diagram"></img>

### Lifecycle of an Action

<img src="./assets/mermaid/ActionStatusFlow.svg" width=1000 alt="Action Status State Diagram"></img>

In [37]:
from madsci.node_module.helpers import action
from madsci.common.types.action_types import ActionResult, ActionSucceeded, ActionRequest, ActionFailed
from madsci.common.types.node_types import NodeStatus

class RobotNodeWithAction(RobotNodeWithUpdates):
    """Define an example robot node with an action."""

    node_status = NodeStatus()

    @action(name="move_joints", description="Move the robot to the specified joint angles")
    def move_joints(self, joint_angles: list[float]) -> ActionResult:
        """
        An example action: moving the robot to a set of joint angles.
        """
        if self.robot_interface is None:
            self.logger.log_error("Robot interface not initialized")
            return ActionFailed(errors="Robot interface not initialized")
        if self.robot_interface.is_moving:
            self.logger.log_error("Robot is already moving")
            return ActionFailed(errors="Robot is already moving")
        if len(joint_angles) != 4:
            self.logger.log_error("Invalid number of joint angles. Expected 4.")
            return ActionFailed(errors="Invalid number of joint angles. Expected 4.")
        self.robot_interface.move_to_joint_angles(joint_angles)
        self.logger.log_info(f"Moved robot to joint angles: {joint_angles}")
        return ActionSucceeded()

example_node = RobotNodeWithAction(node_definition=node_definition)
example_node.start_node(testing=True)

with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    while not client.get_status().ready:
        # Wait for the node to be ready
        time.sleep(0.1)
    request = ActionRequest(
        action_name="move_joints",
        args={"joint_angles": [0.5, 0.5, 0.5, 0.5]},
    )
    # Send the action request to the node
    print(client.send_action(request))
    time.sleep(2)
    print(client.get_state())

2025-04-08 14:16:59.274347 (EventType.LOG_INFO): Connecting to robot 0...

2025-04-08 14:16:59.278646 (EventType.LOG_INFO): Connected to robot 0

2025-04-08 14:16:59.280238 (EventType.LOG_INFO): Startup complete for node example_robot_1.

2025-04-08 14:16:59.385708 (EventType.ACTION_STATUS_CHANGE): action_id='01JRBDB6BRX90C2GJMZYNJ4Z7F' 
status=<ActionStatus.NOT_STARTED: 'not_started'> errors=[] data={} files={} datapoints={} 
history_created_at=datetime.datetime(2025, 4, 8, 14, 16, 59, 385661, 
tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

2025-04-08 14:16:59.387482 (EventType.ACTION_STATUS_CHANGE): action_id='01JRBDB6BRX90C2GJMZYNJ4Z7F' 
status=<ActionStatus.RUNNING: 'running'> errors=[] data={} files={} datapoints={} 
history_created_at=datetime.datetime(2025, 4, 8, 14, 16, 59, 387458, 
tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

...

2025-04-08 14:17:04.394801 (EventType.ACTION_STATUS_CHANGE): action_id='01JRBDB6BRX90C2GJMZYNJ4Z7F' 
status=<ActionStatus.SUCCEEDED: 'succeeded'> errors=[] data={} files={} datapoints={} 
history_created_at=datetime.datetime(2025, 4, 8, 14, 17, 4, 394624, 
tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

ActionResult(
    action_id='01JRBDB6BRX90C2GJMZYNJ4Z7F',
    status=<ActionStatus.SUCCEEDED: 'succeeded'>,
    errors=[],
    data={},
    files={},
    datapoints={},
    history_created_at=datetime.datetime(2025, 4, 8, 14, 17, 4, 394624, tzinfo=TzInfo(-05:00))
)

{'joint_angles': [0.5, 0.5, 0.5, 0.5]}

2025-04-08 14:17:06.624444 (EventType.LOG_INFO): Disconnecting from robot 0...

2025-04-08 14:17:06.626965 (EventType.LOG_INFO): Disconnected from robot 0

In [38]:
# What if we forget an argument?
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    while not client.get_status().ready:
        # Wait for the node to be ready
        time.sleep(0.1)
    request = ActionRequest(
        action_name="move_joints",
        args={},
    )
    # Send the action request to the node
    print(client.send_action(request))

2025-04-08 14:18:47.376492 (EventType.LOG_INFO): Connecting to robot 0...

2025-04-08 14:18:47.384058 (EventType.LOG_INFO): Connected to robot 0

2025-04-08 14:18:47.386432 (EventType.LOG_INFO): Startup complete for node example_robot_1.

2025-04-08 14:18:47.489519 (EventType.ACTION_STATUS_CHANGE): action_id='01JRBDEFY0FCBSMSSY059VYP3S' 
status=<ActionStatus.NOT_STARTED: 'not_started'> errors=[] data={} files={} datapoints={} 
history_created_at=datetime.datetime(2025, 4, 8, 14, 18, 47, 489468, 
tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

2025-04-08 14:18:47.492307 (EventType.NODE_ERROR): message="Missing required arguments for action 'move_joints': 
['joint_angles']" logged_at=datetime.datetime(2025, 4, 8, 14, 18, 47, 492282) error_type='ValueError'

2025-04-08 14:18:47.494634 (EventType.LOG_ERROR): Traceback (most recent call last):
  File 
"/Users/luckierdodge/AD-SDL/MADSci_Examples/.venv/lib/python3.11/site-packages/madsci/node_module/abstract_node_mod
ule.py", line 216, in run_action
    self._check_required_args(action_request)
  File 
"/Users/luckierdodge/AD-SDL/MADSci_Examples/.venv/lib/python3.11/site-packages/madsci/node_module/abstract_node_mod
ule.py", line 764, in _check_required_args
    raise ValueError(
ValueError: Missing required arguments for action 'move_joints': ['joint_angles']

2025-04-08 14:18:47.495967 (EventType.ACTION_STATUS_CHANGE): action_id='01JRBDEFY0FCBSMSSY059VYP3S' 
status=<ActionStatus.FAILED: 'failed'> errors=[Error(message="Missing required arguments for action 'move_joints': 
['joint_angles']", logged_at=datetime.datetime(2025, 4, 8, 14, 18, 47, 495878), error_type='ValueError')] data={} 
files={} datapoints={} history_created_at=datetime.datetime(2025, 4, 8, 14, 18, 47, 495913, 
tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

ActionResult(
    action_id='01JRBDEFY0FCBSMSSY059VYP3S',
    status=<ActionStatus.FAILED: 'failed'>,
    errors=[
        Error(
            message="Missing required arguments for action 'move_joints': ['joint_angles']",
            logged_at=datetime.datetime(2025, 4, 8, 14, 18, 47, 495878),
            error_type='ValueError'
        )
    ],
    data={},
    files={},
    datapoints={},
    history_created_at=datetime.datetime(2025, 4, 8, 14, 18, 47, 495913, tzinfo=TzInfo(-05:00))
)

2025-04-08 14:18:47.500014 (EventType.LOG_INFO): Disconnecting from robot 0...

2025-04-08 14:18:47.500790 (EventType.LOG_INFO): Disconnected from robot 0

In [39]:
# What if the action fails?
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    while not client.get_status().ready:
        # Wait for the node to be ready
        time.sleep(0.1)
    request = ActionRequest(
        action_name="move_joints",
        args={"joint_angles": [0.5, 0.5, 0.5]},
    )
    # Send the action request to the node
    print(client.send_action(request))

2025-04-08 14:19:02.267679 (EventType.LOG_INFO): Connecting to robot 0...

2025-04-08 14:19:02.273461 (EventType.LOG_INFO): Connected to robot 0

2025-04-08 14:19:02.276819 (EventType.LOG_INFO): Startup complete for node example_robot_1.

2025-04-08 14:19:02.382138 (EventType.ACTION_STATUS_CHANGE): action_id='01JRBDEYFD34J74Z5M2765Q6VT' 
status=<ActionStatus.NOT_STARTED: 'not_started'> errors=[] data={} files={} datapoints={} 
history_created_at=datetime.datetime(2025, 4, 8, 14, 19, 2, 382094, 
tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

2025-04-08 14:19:02.383968 (EventType.ACTION_STATUS_CHANGE): action_id='01JRBDEYFD34J74Z5M2765Q6VT' 
status=<ActionStatus.RUNNING: 'running'> errors=[] data={} files={} datapoints={} 
history_created_at=datetime.datetime(2025, 4, 8, 14, 19, 2, 383925, 
tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

2025-04-08 14:19:02.385805 (EventType.LOG_ERROR): Invalid number of joint angles. Expected 4.

2025-04-08 14:19:02.387239 (EventType.ACTION_STATUS_CHANGE): action_id='01JRBDEYFD34J74Z5M2765Q6VT' 
status=<ActionStatus.FAILED: 'failed'> errors=[Error(message='Invalid number of joint angles. Expected 4.', 
logged_at=datetime.datetime(2025, 4, 8, 14, 19, 2, 387190), error_type=None)] data={} files={} datapoints={} 
history_created_at=datetime.datetime(2025, 4, 8, 14, 19, 2, 387198, 
tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'CDT'))

ActionResult(
    action_id='01JRBDEYFD34J74Z5M2765Q6VT',
    status=<ActionStatus.FAILED: 'failed'>,
    errors=[
        Error(
            message='Invalid number of joint angles. Expected 4.',
            logged_at=datetime.datetime(2025, 4, 8, 14, 19, 2, 387190),
            error_type=None
        )
    ],
    data={},
    files={},
    datapoints={},
    history_created_at=datetime.datetime(2025, 4, 8, 14, 19, 2, 387198, tzinfo=TzInfo(-05:00))
)

2025-04-08 14:19:02.391753 (EventType.LOG_INFO): Disconnecting from robot 0...

2025-04-08 14:19:02.392751 (EventType.LOG_INFO): Disconnected from robot 0

In [40]:
# What does the node info look like?
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    while not client.get_status().ready:
        # Wait for the node to be ready
        time.sleep(0.1)
    # Example of a request to the node
    print(client.get_info())

2025-04-08 14:19:13.357666 (EventType.LOG_INFO): Connecting to robot 0...

2025-04-08 14:19:13.363380 (EventType.LOG_INFO): Connected to robot 0

2025-04-08 14:19:13.365886 (EventType.LOG_INFO): Startup complete for node example_robot_1.

NodeInfo(
    node_name='example_robot_1',
    node_id='01JRBCQV3MGHTHNSTF3JNH6NBN',
    node_url=AnyUrl('http://127.0.0.1:8000/'),
    node_description='An example node for controlling our fake robot arm.',
    node_type=<NodeType.DEVICE: 'device'>,
    module_name='example_robot_module',
    module_version=SemanticVersion(major=0, minor=0, patch=1, prerelease=None, build=None),
    capabilities=NodeCapabilities(
        get_info=True,
        get_state=True,
        get_status=True,
        send_action=True,
        get_action_result=True,
        get_action_history=True,
        action_files=True,
        send_admin_commands=True,
        set_config=True,
        get_resources=False,
        get_log=True,
        events=None,
        resources=None,
        admin_commands={
            <AdminCommands.LOCK: 'lock'>,
            <AdminCommands.RESET: 'reset'>,
            <AdminCommands.UNLOCK: 'unlock'>,
            <AdminCommands.SHUTDOWN: 'shutdown'>
        }
    ),
    commands={},
    is_template=False,
    config_defaults={'state_update_interval': 2, 'status_update_interval': 0.5},
    actions={
        'move_joints': ActionDefinition(
            name='move_joints',
            description='Move the robot to the specified joint angles',
            args={
                'joint_angles': ArgumentDefinition(
                    name='joint_angles',
                    description='',
                    argument_type='list[float]',
                    required=True,
                    default=None
                )
            },
            locations={},
            files={},
            results={},
            blocking=False,
            asynchronous=True
        )
    },
    config={
        'status_update_interval': 0.5,
        'state_update_interval': 2.0,
        'event_client_config': None,
        'resource_server_url': None,
        'host': '127.0.0.1',
        'port': 8000,
        'protocol': 'http',
        'robot_number': 0
    },
    config_schema={
        '$defs': {
            'EventClientConfig': {
                'description': 'Configuration for an Event Client.',
                'properties': {
                    'name': {
                        'anyOf': [{'type': 'string'}, {'type': 'null'}],
                        'default': None,
                        'description': 'The name of the event client.',
                        'title': 'Event Client Name'
                    },
                    'event_server_url': {
                        'anyOf': [{'type': 'string'}, {'type': 'null'}],
                        'default': None,
                        'description': 'The URL of the event server.',
                        'title': 'Event Server URL'
                    },
                    'log_level': {
                        'default': 20,
                        'description': 'The log level of the event client.',
                        'title': 'Event Client Log Level',
                        'type': 'integer'
                    },
                    'source': {
                        '$ref': '#/$defs/OwnershipInfo',
                        'description': 'Information about the source of the event client.',
                        'title': 'Source'
                    },
                    'log_dir': {
                        'anyOf': [{'type': 'string'}, {'format': 'path', 'type': 'string'}],
                        'description': 'The directory to store logs in.',
                        'title': 'Log Directory'
                    }
                },
                'title': 'EventClientConfig',
                'type': 'object'
            },
            'OwnershipInfo': {
                'description': 'Information about the ownership of a MADSci object.',
                'properties': {
                    'user_id': {
                        'anyOf': [{'type': 'string'}, {'type': 'null'}],
                        'default': None,
                        'description': 'The

2025-04-08 14:19:13.487588 (EventType.LOG_INFO): Disconnecting from robot 0...

2025-04-08 14:19:13.491483 (EventType.LOG_INFO): Disconnected from robot 0

## Admin Commands

Admin commands provide a standardized interface for common administrative tasks used to control nodes. Unlike actions, admin commands have pre-defined behavior, and the node developer chooses whether and how to implement a specific admin command for a given device in the node. The MADSci Node Standard supports the following admin commands:

- `pause`: pause the current action, if any
- `resume`: resume the current action, if any
- `lock`: refuse to accept new actions
- `unlock`: resume accepting new actions
- `cancel`: cancel the current action
- `reset`: clear errors, reinitialize the node, and reset the status
- `stop`: a more aggressive cancellation--halt immediately and trigger any safety/e-stop of the device
- `shutdown`: disconnect from the device and stop the node's server


In [41]:
# * Admin Command Example: Locking, Unlocking, and Reseting the Node
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    while not client.get_status().ready:
        # Wait for the node to be ready
        time.sleep(0.1)
    print(client.send_admin_command("lock"))
    request = ActionRequest(
        action_name="move_joints",
        args={"joint_angles": [0.5, 0.5, 0.5, 0.5]},
    )
    # Send the action request to the node
    print(client.send_action(request))
    print(client.send_admin_command("unlock"))
    # Send the action request to the node again
    print(client.send_action(request))

    # * Admin Command Example: Resetting the Node
    print(client.send_admin_command("reset"))
    while not client.get_status().ready:
        # Wait for the node to be ready
        time.sleep(0.1)
    time.sleep(3)
    print(client.get_state())

2025-04-08 14:19:59.511034 (EventType.LOG_INFO): Connecting to robot 0...

2025-04-08 14:19:59.516137 (EventType.LOG_INFO): Connected to robot 0

2025-04-08 14:19:59.518584 (EventType.LOG_INFO): Startup complete for node example_robot_1.

2025-04-08 14:19:59.624305 (EventType.LOG_INFO): Node locked

AdminCommandResponse(success=True, errors=[], data=None)

...

2025-04-08 14:20:04.879413 (EventType.LOG_INFO): Connected to robot 0

2025-04-08 14:20:04.882278 (EventType.LOG_INFO): Startup complete for node example_robot_1.

{'joint_angles': [0.0, 0.0, 0.0, 0.0]}

2025-04-08 14:20:07.991370 (EventType.LOG_INFO): Disconnecting from robot 0...

2025-04-08 14:20:07.992402 (EventType.LOG_INFO): Disconnected from robot 0